In [21]:
import pandas as pd
import numpy as np
import json
import warnings
from evaluator import *

# Import data

In [22]:
# file path to data folder
path = './Output'

Metadata (subjectID etc.)

In [23]:
with open(path + '/metadata_stat.txt') as f:
    metadata = f.read()

metadata = json.loads(metadata.replace('\'', '\"').replace('False', 'false').replace('True', 'true')) # doesn't accept other chars

In [24]:
subjects = [meta['subjectID'] for meta in metadata]

# Model

In [25]:
class RandomGuesser:

    def __init__(self, path, variable):
        self.SEED = 42
        np.random.seed(self.SEED)
        self.path = path
        assert variable in (0, 1)
        self.variable = variable

    def load_data(self, indices):
        # load shape
        N = len(indices)
        N_FEATURES = np.load(self.path + '/feature_vector_stat0.npy').shape[0]

        # init
        X = np.empty((N, N_FEATURES))
        y = np.empty(N)

        # load individual datapoints
        for i, index in enumerate(indices):
            X[i, ] = np.load(path + f'/feature_vector_stat{index}.npy', allow_pickle=True)
            y[i, ] = np.load(path + f'/labels_stat{index}.npy', allow_pickle=True)[self.variable]

        return X, y

    def fit(self, train_indices):
        pass

    def predict(self, test_indices):
        return np.random.choice([0, 1], len(test_indices))

# CV

In [26]:
with warnings.catch_warnings():
    warnings.filterwarnings('ignore')

    for variable in (0, 1): # phF, MF
        model = RandomGuesser(path, variable=0)

        scores_strat_group_k_fold = stratified_group_k_fold(path=path,
                                                            groups=subjects,
                                                            model=model,
                                                            folds=5,
                                                            images=False,
                                                            verbose=True,
                                                            variable=variable)

        scores_strat_k_fold = stratified_k_fold(path=path,
                                                model=model,
                                                folds=5,
                                                images=False,
                                                verbose=True,
                                                variable=variable)

        scores_loso = leave_one_subject_out(path=path,
                                            groups=subjects,
                                            model=model,
                                            images=False,
                                            verbose=True,
                                            variable=variable)

Starting stratified group 5-fold for physical fatigue


 Fold 5 F1: 0.7087900279389642: 100%|██████████| 5/5 [00:00<00:00, 78.13it/s]

Performance model:
 accuracy: 0.526 +- 0.054 

 balanced_accuracy: 0.578 +- 0.109 

 f1: 0.564 +- 0.081 

 recall: 0.526 +- 0.054 

 precision: 0.678 +- 0.177 



Starting stratified 5-fold for physical fatigue


 Fold 5 F1: 0.41188892408404604: 100%|██████████| 5/5 [00:00<00:00, 200.03it/s]


...

 Fold 5 F1: 0.481939356939357: 100%|██████████| 5/5 [00:00<00:00, 172.41it/s]  

Performance model:
 accuracy: 0.467 +- 0.026 

 balanced_accuracy: 0.443 +- 0.042 

 f1: 0.486 +- 0.026 

 recall: 0.467 +- 0.026 

 precision: 0.529 +- 0.036 



Starting leave-one-subject-out for mental fatigue


 Fold 21 F1: 0.2: 100%|██████████| 21/21 [00:00<00:00, 177.97it/s]                

Performance model:
 accuracy: 0.511 +- 0.239 

 balanced_accuracy: 0.518 +- 0.243 

 f1: 0.544 +- 0.252 

 recall: 0.511 +- 0.239 

 precision: 0.651 +- 0.313 



# Save scores

In [27]:
path_scores = './Scores'
model_name = 'random_guess'

In [28]:
# stratified 5-fold
with open(f'{path_scores}/strat_5_fold/{model_name}.txt', 'w') as dat:
    dat.write(str(scores_strat_group_k_fold))

In [29]:
# stratified group 5-fold
with open(f'{path_scores}/strat_group_5_fold/{model_name}.txt', 'w') as dat:
    dat.write(str(scores_strat_k_fold))

In [30]:
# LOSO
with open(f'{path_scores}/loso/{model_name}.txt', 'w') as dat:
    dat.write(str(scores_loso))